# NAIP and CHM Spatial Alignment Notebook

## Overview
This notebook performs spatial alignment and resampling between NAIP (National Agriculture Imagery Program) aerial imagery and CHM (Canopy Height Model) data to ensure consistent spatial registration for integrated analysis. The workflow handles potential misalignments in extent, resolution, and pixel registration that can occur between different data sources.

## Purpose and Application
While current classification workflows use NAIP and CHM independently (NAIP for bare earth analysis, CHM for vegetation assessment), this alignment process prepares the data for future integrated analyses that may require both datasets to be spatially synchronized.

## Methodology
The alignment process includes:
1. **Spatial Extent Analysis**: Compare bounding boxes of NAIP and CHM datasets
2. **Resolution Harmonization**: Identify the finer resolution and set as target
3. **Overlap Calculation**: Determine the intersecting spatial area
4. **Resampling and Clipping**: Align both datasets to common extent and resolution
5. **Meadow Boundary Clipping**: Extract data within the study area boundary

## When to Use This Notebook
- **Required**: When NAIP and CHM have different pixel sizes or extents
- **Optional**: When datasets are already well-aligned spatially
- **Future-proofing**: To prepare data for potential multi-sensor analyses

## Data Requirements
- NAIP imagery file (`naip_{site}.tif`)
- Canopy Height Model (`chm_{site}.tif`)
- Meadow boundary polygon (`meadow_extent.geojson`)

## Output Products
- Spatially aligned NAIP and CHM rasters
- Consistent pixel registration and extent
- Common coordinate reference system

---

In [1]:
# System and file operations
import os

# Raster processing and geospatial operations
import rasterio
from rasterio.windows import from_bounds
from rasterio.enums import Resampling

# Numerical and array operations
import numpy as np

# Visualization
import matplotlib.pyplot as plt

# Image processing (optional - for advanced alignment features)
from skimage import io, color, transform
from skimage.feature import SIFT, match_descriptors, plot_matched_features
from skimage.measure import ransac
from skimage.transform import ProjectiveTransform, warp

def get_overlap_bounds(bounds1, bounds2):
    """
    Calculate the overlapping bounding box between two rasters.
    
    Parameters:
    bounds1, bounds2: rasterio.coords.BoundingBox
        Bounding boxes from two raster datasets
        
    Returns:
    tuple: (left, bottom, right, top) of overlapping area
    
    Raises:
    ValueError: If no overlapping area exists
    """
    left = max(bounds1.left, bounds2.left)
    bottom = max(bounds1.bottom, bounds2.bottom)
    right = min(bounds1.right, bounds2.right)
    top = min(bounds1.top, bounds2.top)

    if left < right and bottom < top:
        return (left, bottom, right, top)
    else:
        raise ValueError("No overlapping area between rasters.")

def get_resolution(transform):
    """
    Extract pixel size from raster affine transform.
    
    Parameters:
    transform: rasterio.Affine
        Affine transformation matrix from raster
        
    Returns:
    tuple: (x_resolution, y_resolution) in map units
    """
    return abs(transform.a), abs(transform.e)

def read_and_clip_resample(path, overlap_bounds, target_res):
    """
    Read, clip, and resample a raster to specified bounds and resolution.
    
    Parameters:
    path: str
        Path to input raster file
    overlap_bounds: tuple  
        (left, bottom, right, top) bounding box
    target_res: tuple
        (x_res, y_res) target pixel size
        
    Returns:
    tuple: (data_array, raster_profile)
    """
    with rasterio.open(path) as src:
        # Create window for the overlap area
        window = from_bounds(*overlap_bounds, transform=src.transform)
        transform = src.window_transform(window)

        # Calculate new dimensions based on target resolution
        width = int((overlap_bounds[2] - overlap_bounds[0]) / target_res[0])
        height = int((overlap_bounds[3] - overlap_bounds[1]) / target_res[1])

        # Read and resample data using bilinear interpolation
        data = src.read(
            window=window,
            out_shape=(height, width),
            resampling=Resampling.bilinear
        )

        # Create new transform for the resampled data
        new_transform = rasterio.transform.from_origin(
            overlap_bounds[0], overlap_bounds[3], target_res[0], target_res[1]
        )

        # Update profile with new dimensions and transform
        profile = src.profile
        profile.update({
            "height": height,
            "width": width,
            "transform": new_transform
        })

    return data, profile

In [2]:
# Enable interactive plotting with mpld3 for better visualization in Jupyter
import mpld3
mpld3.enable_notebook()  
print("✓ Interactive plotting enabled for enhanced data visualization")

✓ Interactive plotting enabled for enhanced data visualization


In [3]:
# Study site identifier - change this to match your data folder
# Available sites: "Lacey", "Humbug", "subb"
name = "Lacey"
print(f"Processing alignment for: {name} meadow")

Processing alignment for: Lacey meadow


In [4]:
# Navigate to the study site data directory
# This directory should contain NAIP, CHM, and boundary files
os.chdir(f"../data/{name}")
current_dir = os.getcwd()
print(f"Working directory: {current_dir}")
print(f"Expected files: naip_{name}.tif, chm_{name}.tif, meadow_extent.geojson")

Working directory: c:\Users\jvonnonn\Gitlab_repos\meadows\data\Lacey
Expected files: naip_Lacey.tif, chm_Lacey.tif, meadow_extent.geojson


In [5]:
# Define file paths for all required datasets
naip_path = f"naip_{name}.tif"          # NAIP aerial imagery
dsm_path = f"dsm_{name}.tif"            # Digital Surface Model (optional)
dtm_path = f"dtm_{name}.tif"            # Digital Terrain Model (optional)
chm_path = f"chm_{name}.tif"            # Canopy Height Model  
wm_path = f"wm_{name}.tif"       # Water mask
meadow_extent_path = "meadow_extent.geojson"  # Study area boundary

# Verify all required files exist
required_files = [naip_path, dsm_path, dtm_path, chm_path, wm_path, meadow_extent_path]
missing_files = []

for file_path in required_files:
    if os.path.exists(file_path):
        print(f"✓ Found: {file_path}")
    else:
        print(f"✗ Missing: {file_path}")
        missing_files.append(file_path)
        
if missing_files:
    print(f"\nERROR: Missing {len(missing_files)} required file(s)")
    print("Please ensure all files are present before proceeding.")
else:
    print(f"\n✓ All required files found. Ready to proceed with alignment.")

✓ Found: naip_Lacey.tif
✓ Found: dsm_Lacey.tif
✓ Found: dtm_Lacey.tif
✓ Found: chm_Lacey.tif
✓ Found: wm_Lacey.tif
✓ Found: meadow_extent.geojson

✓ All required files found. Ready to proceed with alignment.


## Spatial Extent Analysis and Overlap Calculation
Analyze the spatial properties of NAIP and CHM datasets to determine overlapping areas and optimal processing parameters.

In [6]:
# Analyze spatial properties of both datasets
print("Analyzing spatial properties of NAIP and CHM datasets...")

with rasterio.open(naip_path) as r1, rasterio.open(chm_path) as r2:
    # Extract bounding boxes
    bounds1 = r1.bounds  # NAIP bounds
    bounds2 = r2.bounds  # CHM bounds
    
    # Extract pixel resolutions
    res1 = get_resolution(r1.transform)  # NAIP resolution
    res2 = get_resolution(r2.transform)  # CHM resolution
    
    print(f"NAIP bounds: {bounds1}")
    print(f"CHM bounds:  {bounds2}")
    print(f"NAIP resolution: {res1[0]:.3f} x {res1[1]:.3f} meters")
    print(f"CHM resolution:  {res2[0]:.3f} x {res2[1]:.3f} meters")

    # Determine the finer resolution as target
    # Use the smallest pixel size in each dimension
    target_res = (min(res1[0], res2[0]), min(res1[1], res2[1]))
    target_res = round(target_res[0], 2), round(target_res[1], 2)
    print(f"Target resolution: {target_res[0]} x {target_res[1]} meters")

    # Calculate overlapping spatial extent
    overlap_bounds = get_overlap_bounds(bounds1, bounds2)
    print(f"Overlapping bounds: {overlap_bounds}")
    
    # Calculate overlap area
    overlap_width = overlap_bounds[2] - overlap_bounds[0]
    overlap_height = overlap_bounds[3] - overlap_bounds[1]
    overlap_area = overlap_width * overlap_height
    print(f"Overlap area: {overlap_area:,.0f} square meters ({overlap_area/10000:.1f} hectares)")

Analyzing spatial properties of NAIP and CHM datasets...
NAIP bounds: BoundingBox(left=-37391.484, bottom=161019.31499999997, right=-35160.683999999994, top=163027.51499999998)
CHM bounds:  BoundingBox(left=-37391.48357426143, bottom=161018.71401353393, right=-35160.08357426143, top=163028.11401353392)
NAIP resolution: 0.600 x 0.600 meters
CHM resolution:  0.600 x 0.600 meters
Target resolution: 0.6 x 0.6 meters
Overlapping bounds: (-37391.48357426143, 161019.31499999997, -35160.683999999994, 163027.51499999998)
Overlap area: 4,479,892 square meters (448.0 hectares)


In [7]:
# Round overlap bounds for cleaner processing
# This helps avoid floating-point precision issues
overlap_bounds = (
    round(overlap_bounds[0], 3), 
    round(overlap_bounds[1], 3), 
    round(overlap_bounds[2], 3), 
    round(overlap_bounds[3], 3)
)
print(f"Rounded overlap bounds: {overlap_bounds}")
print("Bounds format: (left, bottom, right, top)")

Rounded overlap bounds: (-37391.484, 161019.315, -35160.684, 163027.515)
Bounds format: (left, bottom, right, top)


In [8]:
# Adjust overlap bounds to ensure pixel-perfect alignment
# This step ensures that the overlap area dimensions are exactly divisible 
# by the target resolution, preventing sub-pixel misalignment issues

left, bottom, right, top = overlap_bounds
xres, yres = target_res

print("Adjusting bounds for pixel-perfect alignment...")

# Calculate current dimensions in pixels
width = (right - left) / xres
height = (top - bottom) / yres
print(f"Current dimensions: {width:.2f} x {height:.2f} pixels")

# Floor to nearest integer to ensure whole pixels
width_int = int(np.floor(width))
height_int = int(np.floor(height))
print(f"Adjusted dimensions: {width_int} x {height_int} pixels")

# Recalculate bounds to ensure exact divisibility by target resolution
right_adj = left + width_int * xres
top_adj = bottom + height_int * yres

overlap_bounds_adj = (left, bottom, right_adj, top_adj)
print(f"Original bounds:  {overlap_bounds}")
print(f"Adjusted bounds:  {overlap_bounds_adj}")

# Calculate area difference
original_area = (right - left) * (top - bottom)
adjusted_area = (right_adj - left) * (top_adj - bottom)
area_diff = original_area - adjusted_area
print(f"Area reduction: {area_diff:.0f} sq meters ({area_diff/original_area*100:.2f}%)")

Adjusting bounds for pixel-perfect alignment...
Current dimensions: 3718.00 x 3347.00 pixels
Adjusted dimensions: 3717 x 3347 pixels
Original bounds:  (-37391.484, 161019.315, -35160.684, 163027.515)
Adjusted bounds:  (-37391.484, 161019.315, -35161.284, 163027.515)
Area reduction: 1205 sq meters (0.03%)


In [9]:
# Clip and resample both rasters to the common overlap area and resolution
print("Processing NAIP and CHM datasets...")
print("This may take a few moments depending on file sizes...")

# Process NAIP imagery
print("  • Clipping and resampling NAIP imagery...")
raster1_data, profile1 = read_and_clip_resample(naip_path, overlap_bounds_adj, target_res)
print(f"    NAIP output shape: {raster1_data.shape}")

# Process CHM data  
print("  • Clipping and resampling CHM data...")
raster2_data, profile2 = read_and_clip_resample(chm_path, overlap_bounds_adj, target_res)
print(f"    CHM output shape: {raster2_data.shape}")

print("✓ Both datasets successfully processed to common extent and resolution")

Processing NAIP and CHM datasets...
This may take a few moments depending on file sizes...
  • Clipping and resampling NAIP imagery...
    NAIP output shape: (4, 3347, 3716)
  • Clipping and resampling CHM data...
    CHM output shape: (1, 3347, 3716)
✓ Both datasets successfully processed to common extent and resolution


In [10]:
#optional
print("Processing elevation models and water mask...")
raster3_data, profile3 = read_and_clip_resample(dsm_path, overlap_bounds_adj, target_res)
raster4_data, profile4 = read_and_clip_resample(dtm_path, overlap_bounds_adj, target_res)
raster5_data, profile5 = read_and_clip_resample(wm_path, overlap_bounds_adj, target_res)
print(f"    DSM output shape: {raster3_data.shape}")
print(f"    DTM output shape: {raster4_data.shape}")
print(f"    Water mask output shape: {raster5_data.shape}")
print("✓ Elevation models successfully processed to common extent and resolution")

Processing elevation models and water mask...
    DSM output shape: (1, 3347, 3716)
    DTM output shape: (1, 3347, 3716)
    Water mask output shape: (1, 3347, 3716)
✓ Elevation models successfully processed to common extent and resolution


In [11]:
# Save the aligned raster products for future use
print("Saving aligned raster products...")

# Save aligned NAIP imagery
naip_output = "NAIP_clipped_resampled.tif"
with rasterio.open(naip_output, "w", **profile1) as dst:
    dst.write(raster1_data)
print(f"✓ Saved: {naip_output}")

# Save aligned CHM data
chm_output = "chm_clipped_resampled.tif"  
with rasterio.open(chm_output, "w", **profile2) as dst:
    dst.write(raster2_data)
print(f"✓ Saved: {chm_output}")

print(f"\n📁 Aligned products saved to: {os.getcwd()}")
print("These files have identical extents, resolutions, and pixel registration.")

Saving aligned raster products...
✓ Saved: NAIP_clipped_resampled.tif
✓ Saved: chm_clipped_resampled.tif

📁 Aligned products saved to: c:\Users\jvonnonn\Gitlab_repos\meadows\data\Lacey
These files have identical extents, resolutions, and pixel registration.


In [12]:
# Save DSM and DTM aligned products
print("Saving DSM and DTM aligned products...")

dsm_output = "DSM_clipped_resampled.tif"
with rasterio.open(dsm_output, "w", **profile3) as dst:
    dst.write(raster3_data)
print(f"✓ Saved: {dsm_output}")

dtm_output = "DTM_clipped_resampled.tif"
with rasterio.open(dtm_output, "w", **profile4) as dst:
    dst.write(raster4_data)
print(f"✓ Saved: {dtm_output}")

wm_output = "WM_clipped_resampled.tif"
with rasterio.open(wm_output, "w", **profile5) as dst:
    dst.write(raster5_data)
print(f"✓ Saved: {wm_output}")

print(f"\n📁 Aligned DSM/DTM products and water mask saved to: {os.getcwd()}")

Saving DSM and DTM aligned products...
✓ Saved: DSM_clipped_resampled.tif
✓ Saved: DTM_clipped_resampled.tif
✓ Saved: WM_clipped_resampled.tif

📁 Aligned DSM/DTM products and water mask saved to: c:\Users\jvonnonn\Gitlab_repos\meadows\data\Lacey


In [13]:
# Rename saved raster products to format: <prefix>_{name}.tif
# Uses existing variables: name, naip_output, chm_output, dsm_output, dtm_output

outputs = [naip_output, chm_output, dsm_output, dtm_output, wm_output]

for src in outputs:
    if not os.path.exists(src):
        print(f"✗ Missing: {src} — skipping")
        continue

    # derive lowercase prefix from source (e.g., "NAIP_clipped_resampled.tif" -> "naip")
    prefix = os.path.splitext(src)[0].split('_')[0].lower()
    dest = f"{prefix}_{name}.tif"

    try:
        # use os.replace to overwrite existing destination if present
        os.replace(src, dest)
        print(f"✓ Renamed: {src} -> {dest}")
    except Exception as e:
        print(f"✗ Failed to rename {src} -> {dest}: {e}")

✓ Renamed: NAIP_clipped_resampled.tif -> naip_Lacey.tif
✓ Renamed: chm_clipped_resampled.tif -> chm_Lacey.tif
✓ Renamed: DSM_clipped_resampled.tif -> dsm_Lacey.tif
✓ Renamed: DTM_clipped_resampled.tif -> dtm_Lacey.tif
✓ Renamed: WM_clipped_resampled.tif -> wm_Lacey.tif
